In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_qdrant import RetrievalMode, QdrantVectorStore
from qdrant_client.models import Filter, FieldCondition, MatchValue
import os
import re
os.environ["TOKENIZERS_PARALLELISM"] = "false" # Verhindert inkonsistenten Fehler beim Erzeugen von Embeddings (sonst wurde versucht parallel auf die gleiche Ressource zuzugreifen)
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
# Query Präfix für Embeddings der Fragen
encode_kwargs = {'prompt': 'query: '}

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="mixedbread-ai/deepset-mxbai-embed-de-large-v1", encode_kwargs=encode_kwargs)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:datasets:PyTorch version 2.6.0 available.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: mixedbread-ai/deepset-mxbai-embed-de-large-v1


In [4]:
# Qdrant-Client für Token-basierte-Collection
token_collection = "mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks"
qdrant_url = "http://localhost:6333"
token_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=token_collection,
    url=qdrant_url,
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks "HTTP/1.1 200 OK"


In [5]:
# Qdrant-Client für Recursive-Collection
recursive_collection = "mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks"
recursive_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=recursive_collection,
    url=qdrant_url,
)

INFO:httpx:HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks "HTTP/1.1 200 OK"


In [6]:
# 10 Fragen:
fragen = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}

In [7]:
def clean_filename(text: str) -> str:
    text = re.sub(r'[<>:"/\\|?*]', '', text)
    return text.strip()

In [8]:
def save_questions_and_documents_with_relevance(output_dir: str, client: QdrantVectorStore, fragen: dict) -> None:
    os.makedirs(output_dir, exist_ok=True)

    for id, frage in fragen.items():
        frage_filename = f"{id}_{clean_filename(frage)}.txt"  # Dateiname mit ID
        filepath = os.path.join(output_dir, frage_filename)

        found_docs = client.similarity_search_with_score(frage, k=10)

        with open(filepath, "w", encoding="utf-8") as file:
            file.write(f"ID: {id}\n")
            file.write(f"Frage: {frage}\n\n")

            for idx, (doc, score) in enumerate(found_docs, 1):
                file.write(f"Dokument {idx} (Score: {score:.8f}):\n")
                file.write("Metadaten:\n")
                for key, value in doc.metadata.items():
                    file.write(f"  - {key}: {value}\n")

                file.write("\nInhalt:\n")
                file.write(f"{doc.page_content}\n\n")

        print(f"Gespeichert: {filepath}")

In [9]:
save_questions_and_documents_with_relevance("1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz", token_client, fragen)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/3_Welche Fachgebiete sind an der FH vertreten.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 1-mixedbread_token_based-Fragen_mit_Dokumenten_und_Relevanz/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [10]:
save_questions_and_documents_with_relevance("2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz", recursive_client, fragen)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/3_Welche Fachgebiete sind an der FH vertreten.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 2-mixedbread_recursive-Fragen_mit_Dokumenten_und_Relevanz/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [12]:
# Optimale Filter für die Fragen
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )

filters = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

In [13]:
def save_questions_and_documents_with_relevance_with_metadata_filtering(output_dir: str, client: QdrantVectorStore, fragen: dict, filters: dict) -> None:
    os.makedirs(output_dir, exist_ok=True)

    for id, frage in fragen.items():
        frage_filename = f"{id}_{clean_filename(frage)}.txt"  # Dateiname mit ID
        filepath = os.path.join(output_dir, frage_filename)

        found_docs = client.similarity_search_with_score(frage, k=10, filter=filters[id])

        with open(filepath, "w", encoding="utf-8") as file:
            file.write(f"ID: {id}\n")
            file.write(f"Frage: {frage}\n\n")

            for idx, (doc, score) in enumerate(found_docs, 1):
                file.write(f"Dokument {idx} (Score: {score:.8f}):\n")
                file.write("Metadaten:\n")
                for key, value in doc.metadata.items():
                    file.write(f"  - {key}: {value}\n")

                file.write("\nInhalt:\n")
                file.write(f"{doc.page_content}\n\n")

        print(f"Gespeichert: {filepath}")

In [14]:
save_questions_and_documents_with_relevance_with_metadata_filtering("3-mixedbread_token_based-Fragen_mit_Filtern", token_client, fragen, filters)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/3_Welche Fachgebiete sind an der FH vertreten.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 3-mixedbread_token_based-Fragen_mit_Filtern/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [15]:
save_questions_and_documents_with_relevance_with_metadata_filtering("4-mixedbread_recursive-Fragen_mit_Filtern", recursive_client, fragen, filters)

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/3_Welche Fachgebiete sind an der FH vertreten.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/4_Ist eine Beurlaubung während des Studiums möglich.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/5_Welche Studienmodelle werden angeboten.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/6_Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben.txt


INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/7_Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt
Gespeichert: 4-mixedbread_recursive-Fragen_mit_Filtern/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [16]:
# Importe für LLM und SelfQueryRetriever
from langchain_ollama import ChatOllama
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

In [19]:
# LLM initialisieren mit ausreichendem Kontextfenster
llm = ChatOllama(
    model="llama3.1:8b", # llama3-2:3b ist nicht ausreichend. Auch mit dem Modell hier kann es noch zu Problemen kommen, die durch leistungsstärkere LLMs gelöst werden könnten.
    temperature=0, # Keine Zufälligkeit
    num_ctx=16000 # Kontextfenster 
)

In [20]:
docs_description = "Informationen zum Studium an der Fachhochschule Südwestfalen, den Standorten und Studienmodellen."

metadata_field_info = [
    AttributeInfo(
        name="standort",
        description="Standort des Studiengangs. Einer von ['Alle', 'Iserlohn', 'Hagen', 'Soest', 'Meschede', 'Lüdenscheid']",
        type="string",
    ),
    AttributeInfo(
        name="studiengang",
        description="Der Name des Studiengangs. Einer von ['Alle', 'Wirtschaftsinformatik', 'Elektrotechnik']",
        type="string",
    ),
    AttributeInfo(
        name="abschluss",
        description="Der akademische Grad. Einer von ['Alle', 'Bachelor', 'Master']",
        type="string",
    ),
]

In [21]:
search_kwargs = {"k": 10}
token_based_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=token_client,
    document_contents=docs_description,
    metadata_field_info=metadata_field_info,
    use_original_query=True,
    verbose=True,
    search_kwargs=search_kwargs,
)

In [22]:
recursive_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=recursive_client,
    document_contents=docs_description,
    metadata_field_info=metadata_field_info,
    use_original_query=True,
    verbose=True,
    search_kwargs=search_kwargs,
)

In [23]:
def save_questions_with_self_query_retriever(output_dir: str, retriever: SelfQueryRetriever, fragen: dict) -> None:
    os.makedirs(output_dir, exist_ok=True)

    for id, frage in fragen.items():
        frage_filename = f"{id}_{clean_filename(frage)}.txt"
        filepath = os.path.join(output_dir, frage_filename)

        try:
            found_docs = retriever.invoke(frage)  # SelfQueryRetriever-Aufruf

            with open(filepath, "w", encoding="utf-8") as file:
                file.write(f"ID: {id}\n")
                file.write(f"Frage: {frage}\n\n")

                for idx, doc in enumerate(found_docs, 1):
                    file.write(f"Dokument {idx}:\n")
                    file.write("Metadaten:\n")
                    for key, value in doc.metadata.items():
                        file.write(f"  - {key}: {value}\n")

                    file.write("\nInhalt:\n")
                    file.write(f"{doc.page_content}\n\n")

            print(f"Gespeichert: {filepath}")

        except Exception as e:
            print(f"FEHLER: {e}")
            logging.error(f"Fehler bei Frage {id}: {e}")

In [24]:
save_questions_with_self_query_retriever("5-mixedbread_token_based-Fragen_mit_SelfQueryRetriever", token_based_retriever, fragen)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-mixedbread_token_based-Fragen_mit_SelfQueryRetriever/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Mathematik im Bachelorstudiengang Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Soest')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-mixedbread_token_based-Fragen_mit_SelfQueryRetriever/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Fachgebiete an der FH' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


...

Gespeichert: 5-mixedbread_token_based-Fragen_mit_SelfQueryRetriever/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Modul IT-Sicherheit' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-mixedbread_token_based-Fragen_mit_SelfQueryRetriever/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_token_based_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 5-mixedbread_token_based-Fragen_mit_SelfQueryRetriever/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt


In [25]:
save_questions_with_self_query_retriever("6-mixedbread_recursive-Fragen_mit_SelfQueryRetriever", recursive_retriever, fragen)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Master in Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Master'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Meschede')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-mixedbread_recursive-Fragen_mit_SelfQueryRetriever/1_Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Mathematik im Bachelorstudiengang Elektrotechnik' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Soest')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-mixedbread_recursive-Fragen_mit_SelfQueryRetriever/2_Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Fachgebiete an der FH' filter=None limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


...

Gespeichert: 6-mixedbread_recursive-Fragen_mit_SelfQueryRetriever/8_Was muss ich beachten, wenn ich bei einer Prüfung krank bin.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Modul IT-Sicherheit' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Elektrotechnik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-mixedbread_recursive-Fragen_mit_SelfQueryRetriever/9_Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen.txt


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
INFO:langchain.retrievers.self_query.base:Generated Query: query='Bachelorstudiengang Wirtschaftsinformatik in Hagen' filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='abschluss', value='Bachelor'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='studiengang', value='Wirtschaftsinformatik'), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='standort', value='Hagen')]) limit=None
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/mixedbread-ai_deepset-mxbai-embed-de-large-v1_recursive_chunks/points/query "HTTP/1.1 200 OK"


Gespeichert: 6-mixedbread_recursive-Fragen_mit_SelfQueryRetriever/10_Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen.txt
